### Funciones

In [2]:
import polars as pl
import re
import glob

def contar_valores_repetidos(df: pl.DataFrame, columna: str):
    # Número de valores únicos
    num_unicos = df[columna].n_unique()
    print(f"Número de valores únicos en '{columna}': {num_unicos}")

    # Conteo de valores repetidos
    conteo = (
        df.group_by(columna)
          .count()
          .sort("count", descending=True)
    )
    print(f"Conteo de valores repetidos en '{columna}':")
    print(conteo)
    return conteo


def eliminar_filas_invalidas(df: pl.DataFrame) -> pl.DataFrame:
    # Asegurar que las columnas de fecha estén en formato de fecha
    df = df.with_columns([
        pl.col('fecha_convocatoria').cast(pl.Date),
        pl.col('fecha_buenapro').cast(pl.Date),
        pl.col('fecha_consentimiento_bp').cast(pl.Date)
    ])

    # Condiciones de validez
    condiciones_validas = (
        (pl.col('fecha_convocatoria') <= pl.col('fecha_buenapro')) &
        (pl.col('fecha_convocatoria') <= pl.col('fecha_consentimiento_bp')) &
        (pl.col('fecha_buenapro') <= pl.col('fecha_consentimiento_bp'))
    )

    # Contar antes de filtrar
    filas_antes = df.height

    # Aplicar filtro
    df_filtrado = df.filter(condiciones_validas)

    # Contar después de filtrar
    filas_despues = df_filtrado.height
    eliminadas = filas_antes - filas_despues

    print(f"✅ Filas eliminadas por inconsistencia de fechas: {eliminadas}")
    return df_filtrado

def agregar_diferencias_de_fecha(df: pl.DataFrame) -> pl.DataFrame:
    # Asegurar formato fecha
    df = df.with_columns([
        pl.col('fecha_convocatoria').cast(pl.Date),
        pl.col('fecha_buenapro').cast(pl.Date),
        pl.col('fecha_consentimiento_bp').cast(pl.Date)
    ])

    # Crear columnas de diferencias en días
    df = df.with_columns([
        (pl.col('fecha_buenapro') - pl.col('fecha_convocatoria')).dt.days().alias('f_convo_buenapro'),
        (pl.col('fecha_consentimiento_bp') - pl.col('fecha_buenapro')).dt.days().alias('f_buenapro_consentimiento')
    ])

    return df

def agregar_diferencias_de_fecha(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns([
        (pl.col('fecha_buenapro') - pl.col('fecha_convocatoria')).alias('f_convo_buenapro'),
        (pl.col('fecha_consentimiento_bp') - pl.col('fecha_buenapro')).alias('f_buenapro_consentimiento')
    ])

    return df


### Carga de datos

In [12]:
# Lista de rutas de tus archivos
rutas = [
    "Data/CONOSCE_ADJUDICACIONES2018_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2019_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2020_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2021_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2022_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2023_0.xlsx",
    "Data/CONOSCE_ADJUDICACIONES2024_0.xlsx",
]

# Lista para guardar cada DataFrame con año
dataframes = []

for ruta in rutas:
    try:
        df = pl.read_excel(ruta)

        # Extraer el año del nombre del archivo
        match = re.search(r"(\d{4})", ruta)
        year = int(match.group(1)) if match else None

        # Agregar columnas: id y year
        df = df.with_columns([
            pl.concat_str(["descripcion_item", "entidad", "entidad_departamento"], separator=" ").alias("id"),
            pl.lit(year).alias("year")
        ])

        dataframes.append(df)

    except Exception as e:
        print(f"❌ Error leyendo {ruta}: {e}")

# Unir todos los DataFrames
data_total = pl.concat(dataframes, how="vertical")

# Filtrar por monto adjudicado mayor a 5 millones
df_filtrado = data_total.filter(pl.col("monto_adjudicado_item") > 5_000_000)

# Contar valores repetidos en la columna "id"
conteo_ids = df_filtrado.group_by("id").count().sort("count", descending=True)

# Guardar en CSV
conteo_ids.write_csv("valores_repetidos_all.csv")


C:\Users\abrah\AppData\Local\Temp\ipykernel_7596\3906365100.py:41: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  conteo_ids = df_filtrado.group_by("id").count().sort("count", descending=True)


### Filtrado

In [13]:
#nos quedamos con las columnas relevantes
df_filtrado_more = df_filtrado.select([
    'year',
    'codigoconvocatoria',
    'descripcion_item',
    'estado_item',
    'monto_referencial_item',
    'monto_adjudicado_item',
    'moneda',
    'fecha_convocatoria',
    'fecha_buenapro',
    'fecha_consentimiento_bp',
    'tipoprocesoseleccion',
    'objetocontractual',
    'entidad_departamento',
    'entidad'
])

#quitar fechas inválidas
df_limpio = eliminar_filas_invalidas(df_filtrado_more)
# agregar diferencias de fecha
df = agregar_diferencias_de_fecha(df_limpio)

✅ Filas eliminadas por inconsistencia de fechas: 424


In [14]:
df_dias = df.with_columns([
    pl.col("f_convo_buenapro").dt.total_days().cast(pl.Int32),
    pl.col("f_buenapro_consentimiento").dt.total_days().cast(pl.Int32)
])

In [15]:
df_dias

year,codigoconvocatoria,descripcion_item,estado_item,monto_referencial_item,monto_adjudicado_item,moneda,fecha_convocatoria,fecha_buenapro,fecha_consentimiento_bp,tipoprocesoseleccion,objetocontractual,entidad_departamento,entidad,f_convo_buenapro,f_buenapro_consentimiento
i32,i64,str,str,f64,f64,str,date,date,date,str,str,str,str,i32,i32
2018,494521,"""EJECUCI¿N DEL PROYECTO: ¿MEJOR…","""Contratado""",7.9893e6,7.9893e6,"""Soles""",2018-11-19,2018-12-13,2018-12-14,"""Procedimiento Especial de Cont…","""Obra""","""LA LIBERTAD""","""MUNICIPALIDAD PROVINCIAL DE OT…",24,1
2018,466513,"""CONTRATACI¿N DE EJECUCI¿N DE O…","""Contratado""",6.1357e6,6.1357e6,"""Soles""",2018-08-10,2018-11-17,2018-12-03,"""Licitación Pública""","""Obra""","""SAN MARTIN""","""GOBIERNO REGIONAL DE SAN MARTI…",99,16
2018,500600,"""EJECUCI¿N DE OBRA MEJORAMIENTO…","""Contratado""",5896191.2,5896191.2,"""Soles""",2018-10-23,2018-12-20,2018-12-20,"""Licitación Pública""","""Obra""","""UCAYALI""","""GOBIERNO REGIONAL DE UCAYALI S…",58,0
2018,460199,"""ADQUISICION DEL SUMINISTRO DE …","""Contratado""",9.1256e6,9098223.2,"""Soles""",2018-07-17,2018-08-17,2018-09-03,"""Licitación Pública""","""Bien""","""LIMA""","""HOSPITAL CAYETANO HEREDIA""",31,17
2018,504515,"""CONTRATACI¿N DEL SERVICIO DE S…","""Contratado""",5.6953e6,5.6913e6,"""Soles""",2018-12-27,2019-01-15,2019-01-23,"""Adjudicación Simplificada""","""Servicio""","""CUSCO""","""UNIVERSIDAD NACIONAL DE SAN AN…",19,8
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024,1036274,"""SUMINISTRO DE MEZCLA ASF¿LTICA…","""Contratado""",8.733e6,6.16635e6,"""Soles""",2024-07-19,2024-09-05,2024-09-18,"""Licitación Pública""","""Bien""","""LIMA""","""MUNICIPALIDAD METROPOLITANA DE…",48,13
2024,985770,"""MEJORAMIENTO DEL SERVICIO DE H…","""Contratado""",1.4223e7,1.2801e7,"""Soles""",2024-01-26,2024-06-04,2024-06-18,"""Licitación Pública""","""Obra""","""AREQUIPA""","""MUNICIPALIDAD DISTRITAL DE SAC…",130,14
2024,1051964,"""SERVICIO DE ATENCI¿N AMBULATOR…","""Contratado""",0.0,7914628.8,"""Soles""",2024-09-16,2024-10-09,2024-10-10,"""Adjudicación Simplificada-Homo…","""Servicio""","""LIMA""","""FONDO INTANGIBLE SOLIDARIO DE …",23,1


### Exportación

In [16]:
df_dias.write_csv("resultado_all.csv")